In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

folder_path = '/content/drive/My Drive/classification_model'


# Check if the folder exists
if os.path.exists(folder_path):
  # Iterate through each file/folder in the specified path
  for root, dirs, files in os.walk(folder_path):
    for filename in files:
      # Construct the full file path
      file_path = os.path.join(root, filename)
      print(f"Found file: {file_path}")
else:
  print(f"Folder not found: {folder_path}")
categories = ["abdur", "yash", "sanjay"]
img_size = (224, 224)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import cv2
import numpy as np
import os

data = []
labels = []
img_size = (224, 224)
categories = ["abdur", "yash", "sanjay"]

folder_path = '/content/drive/My Drive/classification_model'

for category in categories:
    path = os.path.join(folder_path, category)
    label = categories.index(category)  # Assign a label (0, 1, 2)

    if os.path.exists(path):
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            try:
                img = cv2.imread(img_path)  # Read image
                img = cv2.resize(img, img_size)  # Resize
                img = img / 255.0  # Normalize
                data.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")

# Convert lists to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

print("Data shape:", data.shape)
print("Labels shape:", labels.shape)


Data shape: (56, 224, 224, 3)
Labels shape: (56,)


In [55]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Convert labels to categorical (one-hot encoding)
num_classes = len(categories)
labels = to_categorical(labels, num_classes)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Train set shape: (44, 224, 224, 3) (44, 3)
Test set shape: (12, 224, 224, 3) (12, 3)


In [61]:
#cnn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Output layer
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 879ms/step - accuracy: 0.4531 - loss: 3.4000 - val_accuracy: 0.5833 - val_loss: 0.6809
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 827ms/step - accuracy: 0.6128 - loss: 0.9478 - val_accuracy: 0.5000 - val_loss: 0.9903
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7316 - loss: 0.7040 - val_accuracy: 1.0000 - val_loss: 0.1635
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 825ms/step - accuracy: 0.8954 - loss: 0.1892 - val_accuracy: 1.0000 - val_loss: 0.0272
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0191 - val_accuracy: 1.0000 - val_loss: 0.0057
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 869ms/step - accuracy: 0.9795 - loss: 0.0368 - val_accuracy: 1.0000 - val_loss: 0.0791
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 925ms/step - accuracy: 1.0000 - loss: 0.0258 - val_accuracy: 1.0000 - val_loss: 1.1105e-04
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 802ms/step - accuracy: 0.9899 - loss: 0.0415 - val_accuracy: 1.0000 - val_loss:

In [65]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

cnn_model.fit(datagen.flow(X_train, y_train, batch_size=8), epochs=10, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 882ms/step - accuracy: 0.4658 - loss: 1.1959 - val_accuracy: 1.0000 - val_loss: 0.2011
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7734 - loss: 0.5152 - val_accuracy: 1.0000 - val_loss: 0.2139
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 844ms/step - accuracy: 0.7452 - loss: 0.5399 - val_accuracy: 1.0000 - val_loss: 0.3150
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.7619 - loss: 0.5542 - val_accuracy: 1.0000 - val_loss: 0.3285
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 879ms/step - accuracy: 0.6649 - loss: 0.5163 - val_accuracy: 0.7500 - val_loss: 0.2897
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 921ms/step - accuracy: 0.6392 - loss: 0.8763 - val_accuracy: 1.0000 - val_loss: 0.2075
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 997ms/step - accuracy: 0.8149 - loss: 0.4104 - val_accuracy: 1.0000 - val_loss: 0.1192
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 856ms/step - accuracy: 0.9403 - loss: 0.3950 - val_accuracy: 1.0000 - val_loss: 0.072

In [62]:
#resnet
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

resnet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

resnet_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.3219 - loss: 1.2285 - val_accuracy: 0.3333 - val_loss: 1.0694
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4267 - loss: 1.1661 - val_accuracy: 0.2500 - val_loss: 1.1997
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.3462 - loss: 1.4438 - val_accuracy: 0.2500 - val_loss: 1.2975
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5312 - loss: 1.0661 - val_accuracy: 0.5000 - val_loss: 1.0064
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.3385 - loss: 1.2611 - val_accuracy: 0.5000 - val_loss: 0.9972
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.4246 - loss: 1.1003 - val_accuracy: 0.2500 - val_loss: 1.0922
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.3712 - loss: 1.1493 - val_accuracy: 0.3333 - val_loss: 1.1214
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.3736 - loss: 1.0589 - val_accuracy: 0.2500 - val_loss: 1.0248
Epoch 9/

In [63]:
#mobilenet
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

mobilenet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

mobilenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mobilenet_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 671ms/step - accuracy: 0.4539 - loss: 1.0723 - val_accuracy: 1.0000 - val_loss: 0.1873
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 341ms/step - accuracy: 0.8116 - loss: 0.3302 - val_accuracy: 1.0000 - val_loss: 0.0531
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 275ms/step - accuracy: 0.9730 - loss: 0.1175 - val_accuracy: 1.0000 - val_loss: 0.0444
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 268ms/step - accuracy: 0.9935 - loss: 0.0372 - val_accuracy: 1.0000 - val_loss: 0.0394
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 335ms/step - accuracy: 1.0000 - loss: 0.0265 - val_accuracy: 1.0000 - val_loss: 0.0031
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 276ms/step - accuracy: 1.0000 - loss: 0.0093 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 296ms/step - accuracy: 1.0000 - loss: 0.0215 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 473ms/step - accuracy: 1.0000 - loss: 0.0167 - val_accuracy: 1.0000 - val_loss:

In [64]:
#evaluation
cnn_acc = cnn_model.evaluate(X_test, y_test)[1]
resnet_acc = resnet_model.evaluate(X_test, y_test)[1]
mobilenet_acc = mobilenet_model.evaluate(X_test, y_test)[1]

print(f"CNN Accuracy: {cnn_acc:.2f}")
print(f"ResNet50 Accuracy: {resnet_acc:.2f}")
print(f"MobileNetV2 Accuracy: {mobilenet_acc:.2f}")

best_model = max([(cnn_acc, "CNN"), (resnet_acc, "ResNet50"), (mobilenet_acc, "MobileNetV2")])
print(f"Best model: {best_model[1]} with accuracy {best_model[0]:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - accuracy: 1.0000 - loss: 0.0096
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.9905
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step - accuracy: 1.0000 - loss: 0.0014
CNN Accuracy: 1.00
ResNet50 Accuracy: 0.50
MobileNetV2 Accuracy: 1.00
Best model: MobileNetV2 with accuracy 1.00


In [66]:
mobilenet_model.save("best_model.h5")


In [79]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load the saved model
model = load_model("best_model.h5")

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)
    class_index = np.argmax(prediction)  # Get predicted class index

    categories = ["abdur", "yash", "sanjay"]
    return categories[class_index]  # Return predicted name

# Example usage
result = predict_image("/WIN_20241119_12_33_12_Pro.jpg")

print("Predicted:", result)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'best_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)